In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import calendar

import folium

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import geojson
%matplotlib inline

In [50]:
html_path = '../docs/assets/htmls/'
image_path = '../docs/assets/imgs/'

In [2]:
def convert_timestring_to_time(date_string):
    if date_string is np.nan:
        return date_string
    idx = date_string.find('M')
    if( idx ==-1):
        return datetime.datetime.strptime(date_string, '%H:%M')
    idx -=2
    if date_string[idx] == ' ':
        return datetime.datetime.strptime(date_string, '%I:%M %p')
    return datetime.datetime.strptime(date_string, '%I:%M%p')

In [3]:
df = pd.read_csv("../../school-shootings-data.csv")
df['time'] = df['time'].apply(lambda x: convert_timestring_to_time(x))
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')

In [4]:
df = df[df['long'].notna()]
df.reset_index(drop=True, inplace=True)

# Aggregate by time

In [7]:
# hour
df['hour'] = df['time'].dt.hour
shootings_per_hour = df.loc[:, ['hour']]
shootings_per_hour = shootings_per_hour.groupby(['hour']).aggregate(count=('hour', 'count')).reset_index()
fig = px.bar(shootings_per_hour, x="hour", y="count", height=400)
fig.update_traces(hovertemplate='Hour: %{x} <br> Incidents: %{y}')
fig.update_layout(legend_title="Incidents", 
                    xaxis_range=[0,23],
                    xaxis = dict(
                    tickmode = 'linear',
                    tick0 = 0,
                    dtick = 1
                ),
                    xaxis_title="Hour", 
                    yaxis_title="Count")

plotly.offline.plot(fig, filename=html_path+"shootings_hour.html")

'../docs/assets/htmls/shootings_hour.html'

In [9]:
# month
df['month'] = df['date'].dt.month
shootings_per_month = df.loc[:, ['month']]
shootings_per_month = shootings_per_month.groupby(['month']).aggregate(count=('month', 'count')).reset_index()
fig = px.bar(shootings_per_month, x="month", y="count", height=400)
fig.update_traces(hovertemplate='Month: %{x} <br> Incidents: %{y}')
fig.update_layout(legend_title="Incidents", 
                    xaxis = dict(
                    tickmode = 'linear',
                    tick0 = 1,
                    dtick = 1
                ),
                    xaxis_title="Month", 
                    yaxis_title="Count")

plotly.offline.plot(fig, filename=html_path+"shootings_month.html")

'../docs/assets/htmls/shootings_month.html'

In [11]:
# weekday
cat = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
shootings_per_weekday = df.loc[:, ['day_of_week']]
shootings_per_weekday = shootings_per_weekday.groupby(['day_of_week']).aggregate(count=('day_of_week', 'count')).reindex(cat)
shootings_per_weekday['weekday'] = shootings_per_weekday.index

fig = px.bar(shootings_per_weekday, x="weekday", y="count", height=400)
fig.update_traces(hovertemplate='Weekday: %{x} <br> Incidents: %{y}')
fig.update_layout(  xaxis_title="Weekday", 
                    yaxis_title="Count")
plotly.offline.plot(fig, filename=html_path+"shootings_weekday.html")

'../docs/assets/htmls/shootings_weekday.html'

# Shooter characteristic

In [5]:
races= {'a': 'Asian', 'ai':'American Indian','b':'Black','h':'Hispanic','w':'White'}
def assign_race(race):
    if race is np.nan:
        return "Not known"
    if race in races:
        return races[race]
    return "Not known"

In [6]:
df['gender_shooter1'].isna().sum()
# assign race from dictionary
df['race_ethnicity_shooter'] = df['race_ethnicity_shooter1'].apply(lambda x: assign_race(x))
# reove cells where age of the shooter was not known
df = df[df['age_shooter1'].notna()]
# convert age to int
df['age_shooter1'] = df['age_shooter1'].astype(int)
# separate data for men and women
women = df[df['gender_shooter1'] == 'f']
men = df[df['gender_shooter1'] == 'm']
# aggregate women
women_ethnicity = women.loc[:,['age_shooter1','race_ethnicity_shooter']].value_counts(dropna=False).unstack() 
women_ethnicity = women_ethnicity.fillna(0)
women_ethnicity = women_ethnicity.reset_index()
#aggregate men
men_ethnicity = men.loc[:,['age_shooter1','race_ethnicity_shooter']].value_counts(dropna=False).unstack() 
men_ethnicity = men_ethnicity.fillna(0)
men_ethnicity = men_ethnicity.reset_index()

race_ethnicity_shooter,age_shooter1,Black,Not known,White
0,12,0.0,1.0,0.0
1,14,0.0,1.0,1.0
2,15,1.0,1.0,0.0
3,22,1.0,0.0,0.0
4,28,0.0,2.0,0.0
5,30,0.0,0.0,1.0
6,33,0.0,1.0,0.0


In [46]:
marker_color = {
    "Black":"#fd7f6f",
    "Not known":"#7eb0d5",
    "White": "#b2e061",
    "Hispanic":"#bd7ebe",
    "American Indian":"#ffb55a",
    "Asian": "#ffee65",
}

In [51]:
fig = make_subplots(rows=1, cols=2, specs=[[{"type":"bar"}, {"type": "bar"}]], shared_xaxes=False,
                    shared_yaxes=True, horizontal_spacing=0)

base = np.zeros(shape=women_ethnicity.shape[0])  
base = pd.Series(base)
for col in women_ethnicity.columns[1:]:
    fig.append_trace(go.Bar(name="Ethincity "+col,
                        x=women_ethnicity[col],
                        y=women_ethnicity['age_shooter1'], 
                        hovertemplate='Ethinicity: ' + col + '<br>Count: ' +women_ethnicity[col].map('{:,.0f}'.format), 
                        offsetgroup=0,
                        orientation='h', 
                        width=0.7, 
                        marker_color=marker_color[col],
                        showlegend=False, 
                        base=base), 
                        1, 1) # 1,1 represents row 1 column 1 in the plot grid
    base = base.add(women_ethnicity[col])


base = np.zeros(shape=men_ethnicity.shape[0])  
base = pd.Series(base)
for col in men_ethnicity.columns[1:]:
    fig.append_trace(go.Bar(name="Ethincity "+col,
                        x=men_ethnicity[col],
                        y=men_ethnicity['age_shooter1'], 
                        hovertemplate='Ethinicity: ' + col + '<br>Count: ' +men_ethnicity[col].map('{:,.0f}'.format),
                        offsetgroup=1,
                        orientation='h', 
                        width=0.7, 
                        marker_color=marker_color[col],
                        showlegend=False, 
                        base=base), 
                        1, 2) # 1,1 represents row 1 column 1 in the plot grid
    base = base.add(men_ethnicity[col])

fig.update_xaxes(showticklabels=False,title_text="Female", row=1, col=1, range=[20,0])
fig.update_xaxes(showticklabels=False,title_text="Male", row=1, col=2)

fig.update_layout(title_text="Age of the shooter with ethnicity distinguished", 
                  width=800, 
                  height=700,
                  title_x=0.9,
                  xaxis1={'side': 'top'},
                  xaxis2={'side': 'top'},
                  barmode='relative')

plotly.offline.plot(fig, filename=html_path+"shooter_age_gender_distribution.html")
fig.show()

# Shootings per year

In [30]:

shootings_per_year = df.groupby(['year']).agg(
     sum_killed = ('killed','sum'),
     sum_injured = ('injured','sum'),
     sum=('year', 'count')
     ).reset_index()


,year,sum_killed,sum_injured,sum
0,1999,14,35,7
1,2000,4,3,12
2,2001,5,22,13
3,2002,1,7,5
4,2003,5,8,12


In [ ]:
fig = go.Figure(
    data = [
        go.Bar(x=shootings_per_year['year'], y=shootings_per_year['sum_injured'], offsetgroup=0, name='People injured', hovertemplate='Year: %{x} <br> Injured: %{y}', hovertextsrc=''),
        go.Bar(x=shootings_per_year['year'], y=shootings_per_year['sum_killed'], offsetgroup=0, base=shootings_per_year['sum_injured'],  name='People killed', hovertemplate='Year: %{x} <br> Killed: %{y}'),
        go.Line(x=shootings_per_year['year'], y=shootings_per_year['sum'], name='All incidents', hovertemplate='Year: %{x} <br> Incidents count: %{y}'),
    ]
)
fig.update_layout(width=1000, xaxis_title="Year", yaxis_title="Count")
plotly.offline.plot(fig, filename=html_path+"shootings_year.html")

# Plot student's ethnicity distribution in schools

In [32]:
def plot_student_race_distribution(races, column_names, school_name, file_path):
    races = races.fillna(0)
    fig = px.pie(values=races.values[0], names=column_names, title='Population of '+school_name, width=700)
    fig.update_layout(legend_title="Ethnicity")
    fig.update_traces(hovertemplate='Ethnicity: %{label} <br>  Count: %{value}')
    plotly.offline.plot(fig, filename=file_path)

In [33]:
df.reset_index(drop=True, inplace=True)

In [ ]:
column_names = ['White', 'Black', 'Hispanic', 'Asian',
       'American/Indian/Alaska native', 'Hawaiian native/Pacific islander']
df['ethnicity_diagram_path'] = np.nan
for i in range(0,df.shape[0]):
    races = df.loc[[i], ['white', 'black', 'hispanic', 'asian',
       'american_indian_alaska_native', 'hawaiian_native_pacific_islander']]
    races = races._convert(numeric=True)
    schoolname = df.iloc[i]['school_name']
    filepath = './ethinicity_diagrams/' + schoolname.replace('/', '_') +'_ethinicity.html'
    filepathtosave= html_path + 'ethinicity_diagrams/' + schoolname.replace('/', '_') +'_ethinicity.html'
    plot_student_race_distribution(races, column_names, df.iloc[i]['school_name'], filepathtosave)
    df.at[i,'ethnicity_diagram_path']= filepath

# Map with student's ethnicity in school

In [ ]:
def generate_color(killed, injured):
    if killed ==0 and injured==0:
        return 'blue'
    if killed ==0 :
        return 'gray'
    return 'crimson'

def calculate_circle_radius(max_affected, current_affected):
    return math.log(current_affected/ max_affected * 100 + 1.) * 2 + 4

In [ ]:
max_killed_and_injured = max(df['killed']) + max(df['injured'])
# Make an empty map
m = folium.Map(location=[40,-100], tiles="OpenStreetMap", zoom_start=4.5)
for i in range(0,df.shape[0]):
   killed_and_injured = df.iloc[i]['killed'] + df.iloc[i]['injured']
   # calculate radius
   radius = calculate_circle_radius(max_killed_and_injured, killed_and_injured)
   # prepare html text to embeed in map
   html="""
    <iframe src=\"""" + df['ethnicity_diagram_path'][i] + """\" width="850" height="400"  frameborder="0">    
    """
   diagram = folium.Popup(folium.Html(html, script=True))
   folium.CircleMarker(
      location=[df.iloc[i]['lat'], df.iloc[i]['long']],
      radius=radius,
      popup=diagram,
      color=generate_color(df.iloc[i]['killed'], df.iloc[i]['injured']),
      fill=True,
      fill_color=generate_color(df.iloc[i]['killed'], df.iloc[i]['injured'])
   ).add_to(m)

# Save the map
m.save(html_path+'shootings_details.html')


# Gun ownership

In [ ]:
file_name = '../../TL-354-State-Level Estimates of Household Firearm Ownership.xlsx'
sheet_to_df_map = pd.read_excel(file_name, sheet_name="State-Level Data & Factor Score")
gun_ownership = sheet_to_df_map.groupby(['STATE']).aggregate(
    average_ownership = ('HFR', 'mean')
).reset_index()
# convert rate to %
gun_ownership['average_ownership'] = gun_ownership['average_ownership'] * 100
# get shootings by state
school_shootings_by_state = df.groupby('state').aggregate( killed = ('killed', 'sum'), injured= ('injured', 'sum')).reset_index()

In [91]:
with open('../../us-states.json') as f:
    us_state_map = geojson.load(f)

In [147]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

# generate abbreviations for states
school_shootings_by_state['state_abb'] = school_shootings_by_state['state'].apply(lambda x: us_state_to_abbrev[x])
gun_ownership['state_abb'] = gun_ownership['STATE'].apply(lambda x: us_state_to_abbrev[x])
# generate tooltip text
gun_ownership['text'] = gun_ownership.apply(lambda row: f"{row['STATE']} <br> Average % of households that own a gun: {round(row['average_ownership'],2)} %", axis=1)

In [172]:
layout = dict(font=dict(size=8), geo=dict(scope="usa", projection=dict(type="albers usa")))
data = []

data.append(go.Choropleth(locations=gun_ownership['state_abb'], z=gun_ownership['average_ownership'], locationmode="USA-states", colorscale="bluered", text=gun_ownership["text"], hoverinfo='text'))
data.append(go.Scattergeo(locations=school_shootings_by_state['state_abb'], text=school_shootings_by_state['killed'], locationmode="USA-states", mode="text", hoverinfo='skip'))

fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
# test
choropleth = px.choropleth_mapbox(gun_ownership, geojson=us_state_map, locations='STATE', featureidkey='properties.name', color='average_ownership',
                           color_continuous_scale="Viridis",
                           range_color=(0, 100),
                           zoom=2.5, center = {"lat": 37.770722, "lon": -90.425583},
                           mapbox_style="carto-positron",
                           opacity=0.7,
                           labels={'average_ownership':'Average % of gun ownership in household', "STATE": "State"}
                          )
killed = px.scatter_mapbox(school_shootings_by_state, geojson=us_state_map, locations='state', featureidkey='properties.name', text=school_shootings_by_state['killed'])
fig = go.Figure(data=choropleth)
# fig.append_trace(
#     px.scatter_geo(school_shootings_by_state, geojson=us_state_map, locations='state', featureidkey='properties.name', text=school_shootings_by_state['killed']), 1, 1
# )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()